In [1]:
!pip install -q transformers peft datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.4 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# ✅ Upload ZIP bundle with training script and data
from google.colab import files
uploaded = files.upload()

Saving mistralai_invoice_mapping_finetune_bundle.zip to mistralai_invoice_mapping_finetune_bundle.zip


In [ ]:
# ✅ Unzip the bundle
!unzip -o mistralai_invoice_mapping_finetune_bundle.zip

Archive:  mistralai_invoice_mapping_finetune_bundle.zip
  inflating: finetune_tinyllama.py   
  inflating: train_invoice_sow_10000.jsonl  


In [ ]:
!nvidia-smi

Tue Aug 12 04:31:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             44W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# ✅ Run training
!python fine_tune_mistralai_mapping_auto_eval.py --train_invoice_sow_sample.jsonl --num_train_epochs=3

2025-08-12 22:07:39.963102: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 22:07:39.980679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755036460.002276    4956 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755036460.008727    4956 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755036460.024986    4956 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.make_archive("output_lora_mistralai_invoice_500", "zip", "mistral_lora_map_invoice_500")
from google.colab import files
files.download("output_lora_mistralai_invoice_500.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install "transformers>=4.41.0" "peft>=0.11.1" "accelerate>=0.30.0" "bitsandbytes>=0.43.1" datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 9.8 MB/s eta 0:00:00


In [ ]:
!pip install -U trl>=0.8.6

In [ ]:
!python fine_tune_mistralai_mapping_auto_eval.py \
  --train_file train_invoice_sow_sample.jsonl \
  --output_dir ./mistral_lora_map_invoice_500 \
  --eval_ratio 0 \
  --num_train_epochs 2 \
  --per_device_train_batch_size 2 \
  --grad_accum 8 \
  --learning_rate 1e-4 \
  --warmup_ratio 0.1 \
  --bf16 \
  --sample_print_steps 200

2025-08-12 21:59:59.562599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755035999.604987   12644 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755035999.618366   12644 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755035999.657702   12644 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755035999.657742   12644 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755035999.657751   12644 computation_placer.cc:177] computation placer alr

In [ ]:
import shutil
shutil.make_archive("mistral_lora_mapping_training", "zip", "mistral_lora_map")
from google.colab import files
files.download("mistral_lora_mapping_training.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#inference using sample
import torch, json, re
from peft import PeftModel
import os
os.environ.setdefault("BITSANDBYTES_NOWELCOME", "1")
os.environ.setdefault("LD_LIBRARY_PATH", "/usr/lib64-nvidia")
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
ADAPTER_DIR = "/content/mistral_lora_map_invoice_500"       # from training output
MAX_NEW = 768

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb)
model = PeftModel.from_pretrained(base, ADAPTER_DIR).eval()

def wrap_inst(s): return f"<s>[INST] {s.strip()} [/INST]"

def extract_first_json(s: str):
    m = re.search(r"<json>\s*(\{.*?\})\s*</json>", s, flags=re.S|re.I)
    if m:
        try: return json.loads(m.group(1))
        except: pass
    start = s.find("{"); end = s.rfind("}")
    if start!=-1 and end!=-1 and end>start:
        try: return json.loads(s[start:end+1])
        except: pass
    return None

def infer_invoice(doc_text: str):
    instr = (
        "You are a strict JSON extractor for INVOICE documents.\n"
        "Return ONLY one minified JSON object between <json> and </json>. No markdown, no explanations.\n"
        "Fields: title, invoice_id, bill_to, vendor, date, amount, "
        "supplier_information, period, terms, tax, insurance, due_date, payment_method, "
        "additional_text, line_items.\n"
        "Rules:\n"
        "- If a field is not present, use \"Missing\".\n"
        "- If invoice_title is missing, set it to \"Invoice\".\n"
        "- Use \"Remit To\" as vendor if available; if vendor name missing, use business_id or client_id.\n"
        "- Split vendor and bill_to into name, address, phone, email if possible.\n"
        "- Use additional_text for anything not covered.\n"
        "- Output MUST start with '{' and end with '}'."
    )
    prompt = wrap_inst(f"{instr}\n\nDOCUMENT:\n{doc_text}\n\nPlease output:\n<json>\n{{}}\n</json>")

    enc = tok(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    input_len = enc.input_ids.shape[-1]
    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=MAX_NEW,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            top_k=0,
            repetition_penalty=1.02,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.eos_token_id,
        )
    tail = out[0][input_len:]
    text = tok.decode(tail, skip_special_tokens=True)
    return extract_first_json(text), text

# tiny test
doc = "Invoice # TB32\nRemit To: Panacea Direct Inc, Jersey City, NJ\nBill To: Netwoven Inc\nDate: 2025-05-01\nTotal Amount: $22,560.00\nTerms: Net 30"
js, raw = infer_invoice(doc)
print(js)
print(raw[:800])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


None
<json>{"title":"Invoice","invoice_id":"TB32","bill_to":{"name":"Netwoven Inc","address":"Missing","phone":"Missing","email":"Missing"},"vendor":{"name":"Panacea Direct Inc","address":"Jersey City, NJ","phone":"Missing","email":"Missing"},"date":"2025-05-01","amount":22560.0,"supplier_information":{"missing"},"period":{"start":"Missing","end":"Missing"},"terms":"Net 30","tax":{"missing},"insurance":{"missing},"due_date":"Missing","payment_method":"Missing","additional_text":"Missing","line_items":{"missing}}</json>


In [5]:
#inference using pdf
# %%capture
!pip -q install "transformers>=4.41.0" "peft>=0.11.1" "accelerate>=0.30.0" "bitsandbytes>=0.43.1" pypdf
import re, json, torch
from pypdf import PdfReader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# === CONFIG ===
BASE_MODEL   = "mistralai/Mistral-7B-Instruct-v0.2"
ADAPTER_DIR  = "/content/sample_data/"  # your trained LoRA output dir
MAX_NEW      = 768
MAX_DOC_CHARS = 18000  # cap input text to keep within context

# === Load model ===
bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb)
model = PeftModel.from_pretrained(base, ADAPTER_DIR).eval()

def wrap_inst(s: str) -> str:
    return f"<s>[INST] {s.strip()} [/INST]"

def read_pdf_text(path: str) -> str:
    r = PdfReader(path)
    parts = []
    for i, page in enumerate(r.pages):
        try:
            t = page.extract_text() or ""
            # Keep page markers lightly to avoid <INST> confusion
            parts.append(f"\n--- Page {i+1} ---\n{t}")
        except Exception:
            continue
    text = "\n".join(parts)
    # sanitize stray bracket sequences that might confuse chat template
    text = text.replace("[/INST]", " ").replace("[INST]", " ")
    return text[:MAX_DOC_CHARS]

def build_invoice_prompt(doc_text: str) -> str:
    instr = (
        "You are a strict JSON extractor for INVOICE documents.\n"
        "Return ONLY one minified JSON object between <json> and </json>. No markdown, no explanations.\n"
        "Fields: title, invoice_id, bill_to, vendor, date, amount, "
        "supplier_information, period, terms, tax, insurance, due_date, payment_method, "
        "additional_text, line_items.\n"
        "Rules:\n"
        "- If a field is not present, set it to \"Missing\".\n"
        "- If invoice_title or title is missing, set it to \"Invoice\".\n"
        "- line_items can have more than one line. Please extract all fields from line. unitprice may show as 2 words unit and price. QTY or qty or quantity is quantity.\n"
        "- \"additional notes\"  may refer to additional_text.\n"
        "- Use \"Remit To\" as vendor if available; if vendor name missing, use business_id or client_id.\n"
        "- Split vendor and bill_to into name, address, phone, email if possible.\n"
        "- Use additional_text for any content not covered by fields.\n"
        "- Output MUST start with '{' and end with '}' and be wrapped in <json>...</json>."
    )
    return wrap_inst(f"{instr}\n\nDOCUMENT:\n{doc_text}\n\nPlease output:\n<json>\n{{}}\n</json>")

def extract_first_json(s: str):
    m = re.search(r"<json>\s*(\{.*?\})\s*</json>", s, flags=re.S|re.I)
    if m:
        try: return json.loads(m.group(1))
        except: pass
    # fallback: best-effort brace slice
    start = s.find("{"); end = s.rfind("}")
    if start!=-1 and end!=-1 and end>start:
        try: return json.loads(s[start:end+1])
        except: pass
    return None

def infer_invoice_from_pdf(pdf_path: str):
    doc_text = read_pdf_text(pdf_path)
    prompt = build_invoice_prompt(doc_text)
    enc = tok(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    input_len = enc.input_ids.shape[-1]
    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=MAX_NEW,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            top_k=0,
            repetition_penalty=1.02,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.eos_token_id,
        )
    tail = out[0][input_len:]
    text = tok.decode(tail, skip_special_tokens=True)
    parsed = extract_first_json(text)
    return parsed, text

# === Upload + test (Colab) ===
from google.colab import files
print("Upload 1+ invoice PDFs …")
uploaded = files.upload()
for name, _ in uploaded.items():
    js, raw = infer_invoice_from_pdf(name)
    print("\n==== File:", name, "====")
    if js is None:
        print("Failed to parse JSON. Raw tail:\n", raw[-1200:])
    else:
        print(json.dumps(js, indent=2))


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Upload 1+ invoice PDFs …


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Saving TB32.pdf to TB32.pdf

==== File: TB32.pdf ====
{
  "title": "Invoice",
  "invoice_id": "TB32",
  "bill_to": {
    "name": "NETWOVEN INC",
    "address_lines": [
      "4000 PIMLICO DR STE 114 103",
      "PLEASANTON, CA 945883474",
      "USA"
    ],
    "phone": "(408) 666-2533"
  },
  "vendor": {
    "name": "Panacea Direct Inc",
    "address_lines": [
      "Journal Square",
      "4th Floor Suite#459",
      "Jersey City, NJ 07306",
      "US"
    ],
    "phone": "(412) 983-2222"
  },
  "date": "Invoice Date",
  "terms_days": 30,
  "due_date": "05/31/2025",
  "line_items": [
    {
      "line_no": 1,
      "description": "Ayan Thaddeus Shaw Consulting Services for AMD",
      "qty": 160,
      "unit_price": 141.0,
      "subtotal": 22560.0
    }
  ],
  "subtotal": 22560.0,
  "tax": 0.0,
  "insurance": "Missing",
  "service_period": {
    "start": "3/29/2025",
    "end": "4/25/2025"
  },
  "total_amount": 22560.0,
  "balance_due": 22560.0,
  "payment_method": "Missing",
  "ad

In [ ]:
import torch, gc

# 1) Move big models off GPU then drop references
try:
    model.to("cpu")
except Exception:
    pass
for obj in ["model", "base", "peft_model", "pipe", "optimizer", "dataloader"]:
    if obj in globals():
        globals()[obj] = None

# 2) Collect Python garbage
gc.collect()

# 3) Tell PyTorch to release cached VRAM back to the driver
torch.cuda.empty_cache()
torch.cuda.ipc_collect()   # helps in multi-process/IPC scenarios


In [ ]:
!python fine_tune_mistralai_mapping_with_eval.py \
  --train_file train_invoice_sow_sample.jsonl \
  --output_dir ./mistral_lora_map_udated \
  --eval_ratio 0 \
  --num_train_epochs 2 \
  --per_device_train_batch_size 2 \
  --grad_accum 8 \
  --learning_rate 1e-4 \
  --warmup_ratio 0.1 \
  --bf16 \
  --sample_print_steps 200

2025-08-10 20:31:52.100933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754857912.122068   62393 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754857912.128599   62393 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754857912.145446   62393 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754857912.145475   62393 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754857912.145478   62393 computation_placer.cc:177] computation placer alr